In [1]:
#Packages:
import pandas as pd
import string
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine, text

In [19]:
#maximum number of rows to display
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 500)

### Classes

In [3]:
class DbEngine():
    def __init__(self, user, password, ip, port, db):
        self.user = user
        self.password = password
        self.ip = ip
        self.port = port
        self.db = db
    
    def connect(self):
        return create_engine('postgresql://{}:{}@{}:{}/{}' \
                                .format(self.user, 
                                        self.password,
                                        self.ip,
                                        self.port,
                                        self.db), max_overflow=20)

In [4]:
class Queries():
    def __init__(self, engine):
        self.engine = engine
    
    def run(self, sql):
        result = self.engine.connect().execution_options(isolation_level="AUTOCOMMIT").execute((text(sql)))
        return pd.DataFrame(result.fetchall(), columns=result.keys())
    
    def insert(self, sql):
        return self.engine.connect().execution_options(isolation_level="AUTOCOMMIT").execute((text(sql)))

In [5]:
class ETL():
    def __init__(self, query_obj):
        self.queries = query_obj
        
    def inmate_static_info(self, data):
        ############## etnic recognition
        data_people = data[['INTERNOEN', 'GENERO', 'PAIS_INTERNO', 'REINCIDENTE', 
                            'ANO_NACIMIENTO', 'ESTADO_CIVIL','NIVEL_EDUCATIVO', 'CONDIC_EXPECIONAL']] \
                      .drop_duplicates(subset = ["INTERNOEN"])
        data_people = data_people.reset_index(drop=True)
        data_people['CONDIC_EXPECIONAL'] = data_people['CONDIC_EXPECIONAL'].fillna('NINGUNO')
        
        etnic_recognition = self.queries.run("""select * from reconocimiento_etnico""")
        data_people['reconocimiento_etnico'] = 'NINGUNO'
        for i in etnic_recognition.index:
            data_people.loc[(data_people['CONDIC_EXPECIONAL'].str.contains(etnic_recognition['nombre'].values[i])),
                            'reconocimiento_etnico'] =  etnic_recognition['nombre'].values[i]
            
        ############## foreign
        data_people['extranjero'] = 'N'
        data_people.loc[(data_people['CONDIC_EXPECIONAL'].str.contains('EXTRANJEROS')),'extranjero'] =  'S'
        
        ############## sexual diversity
        sexual_dive = self.queries.run("""select * from diversidad_sexual""")
        data_people['diversidad_sexual'] = 'N'
        for i in sexual_dive.index:
            data_people.loc[(data_people['CONDIC_EXPECIONAL'].str.contains(sexual_dive['nombre'].values[i])),
                            'diversidad_sexual'] =  'S'
    
        personas_genero = self.queries.run("""select * from persona where diversidad_sexual = 2""")
        for i in personas_genero.index:
            for j in sexual_dive.index:
                if sexual_dive['nombre'].values[j] in personas_genero['condicion_exepcional'].values[i]:
                    query = str(personas_genero['id_persona'].values[i]) + ', ' \
                                + str(sexual_dive['id_diversidad_sexual'].values[j])
                    self.queries.insert('INSERT INTO public.persona_diversidad_sexual (id_persona, id_diversidad_sexual) VALUES('+query+');')
        
        ############## people
        data_people.to_sql('personas_tmp', con=self.queries.engine)
        self.queries.run('select * from personas_tmp limit 5')
        self.queries.run('SELECT public.tcompararpersonas();')
        
    def inmate_variable_info(self, data):
        data_reg= data[['INTERNOEN', 'GENERO','DELITO','ESTADO_INGRESO','FECHA_CAPTURA',
                        'FECHA_INGRESO','ESTABLECIMIENTO','TENTATIVA','SUBTITULO_DELITO',
                        'AGRAVADO', 'CALIFICADO','FECHA_SALIDA','EDAD','DEPARTAMENTO', 'CIUDAD',
                        'ACTIVIDADES_TRABAJO', 'ACTIVIDADES_ESTUDIO', 'ACTIVIDADES_ENSEÑANZA',
                        'HIJOS_MENORES', 'CONDIC_EXPECIONAL','ESTADO','SITUACION_JURIDICA']]
        
        ############## No exceptional condition
        data_reg['CONDIC_EXPECIONAL'] = data_reg['CONDIC_EXPECIONAL'].fillna('NINGUNO')

        ############## Madre Gestantes
        data_reg['madre_gestante'] = 'NA'
        data_reg.loc[(data_reg['GENERO'].str.contains('FEMENINO')),'madre_gestante'] = 'N'
        data_reg.loc[(data_reg['CONDIC_EXPECIONAL'].str.contains('MADRE GESTANTE')),'madre_gestante'] =  'S'

        data_reg['madre_lactante'] = 'NA'
        data_reg.loc[(data_reg['GENERO'].str.contains('FEMENINO')),'madre_lactante'] = 'N'
        data_reg.loc[(data_reg['CONDIC_EXPECIONAL'].str.contains('MADRE LACTANTE')),'madre_lactante'] =  'S'

        ############## Discapacitad
        data_reg['discapacidad'] = 'N'
        data_reg.loc[(data_reg['CONDIC_EXPECIONAL'].str.contains('CON DISCAPACIDAD')),'discapacidad'] =  'S'

        ############## Adulto Mayor
        data_reg['adulto_mayor'] = 'N'
        data_reg.loc[(data_reg['CONDIC_EXPECIONAL'].str.contains('ADULTO MAYOR')),'adulto_mayor'] =  'S'
        
        ############## registro
        data_reg.to_sql('registros_tmp', con=self.queries.engine)
        self.queries.run('select * from registros_tmp limit 5')
        self.queries.run('SELECT public.tcompararreg();')

### Insert Data

#### Loading Data

In [6]:
# data reading from excel
data = pd.read_excel('data/ReincidenciaPospenadosNal201011Junio2020Rev.xlsx', skiprows = 6)
data.count()

INTERNOEN                142899
DELITO                   142899
TITULO_DELITO            142899
SUBTITULO_DELITO         142899
TENTATIVA                142899
                          ...  
ESTABLECIMIENTO          142899
DEPTO_ESTABLECIMIENTO    140108
MPIO_ESTABLECIMIENTO     140108
REGIONAL                 142899
ESTADO                   142899
Length: 32, dtype: int64

In [7]:
data.columns

Index(['INTERNOEN', 'DELITO', 'TITULO_DELITO', 'SUBTITULO_DELITO', 'TENTATIVA',
       'AGRAVADO', 'CALIFICADO', 'FECHA_INGRESO', 'FECHA_SALIDA',
       'FECHA_CAPTURA', 'SITUACION_JURIDICA', 'ANO_NACIMIENTO', 'EDAD',
       'GENERO', 'ESTADO_CIVIL', 'PAIS_INTERNO', 'DEPARTAMENTO', 'CIUDAD',
       'REINCIDENTE', 'ESTADO_INGRESO', 'ACTIVIDADES_TRABAJO',
       'ACTIVIDADES_ESTUDIO', 'ACTIVIDADES_ENSEÑANZA', 'NIVEL_EDUCATIVO',
       'HIJOS_MENORES', 'CONDIC_EXPECIONAL', 'CODIGO_ESTABLECIMIENTO',
       'ESTABLECIMIENTO', 'DEPTO_ESTABLECIMIENTO', 'MPIO_ESTABLECIMIENTO',
       'REGIONAL', 'ESTADO'],
      dtype='object')

#### ETL

In [8]:
db_engine = DbEngine(user = 'team77', 
                     password = 'mintic2020.',
                     ip = '172.17.0.2', 
                     port = '5432', 
                     db = 'minjusticia')
engine = db_engine.connect()

In [9]:
queries = Queries(engine)

In [10]:
etl = ETL(queries)

In [11]:
etl.inmate_static_info(data)
etl.inmate_variable_info(data)

/opt/conda/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/envs/nlp/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand

### Joining Data

In [20]:
df = queries.run('select * from persona left join registro on persona.id_persona = registro.persona_id_persona')
df

,id_persona,internoen,genero,nacionalidad,reincidente,anio_nacimiento,estado_civil,nivel_educativo,reconocimiento_etnico,diversidad_sexual,extranjero,condicion_exepcional,persona_id_persona,delito_id_delito,estado_ingreso,id_registro,fecha_captura,fecha_ingreso,establecimiento,tentativa,agravado,calificado,fecha_salida,edad,municipio_id_municipio,actividades_estudio,actividades_trabajo,actividades_enseñanza,hijos_menores,condicion_excepcional,estado_id_estado,situacion_juridica,madre_gestante,madre_lactante,discapacidad,adulto_mayor
0,45103,0008649DEF36F510807D0AB58A84CF74E6083FBF,2,22,2,1979,7,2,5,1,1,NINGUNO,45103,145,4,12,2020-02-04,2019-09-02,14,1,2,1,None,40,1,1,2,1,2,NINGUNO,1,1,3,3,1,1
1,45103,0008649DEF36F510807D0AB58A84CF74E6083FBF,2,22,2,1979,7,2,5,1,1,NINGUNO,45103,42,4,13,2020-02-04,2019-09-02,14,1,1,1,None,40,1,1,2,1,2,NINGUNO,1,1,3,3,1,1
2,45103,0008649DEF36F510807D0AB58A84CF74E6083FBF,2,22,2,1979,7,2,5,1,1,NINGUNO,45103,42,2,14,2016-01-22,2016-01-29,16,1,1,1,2017-02-02,40,1,1,2,1,2,NINGUNO,2,1,3,3,1,1
3,45103,0008649DEF36F510807D0AB58A84CF74E6083FBF,2,22,2,1979,7,2,5,1,1,NINGUNO,45103,145,2,15,2016-01-22,2016-01-29,16,1,1,1,2017-02-02,40,1,1,2,1,2,NINGUNO,2,1,3,3,1,1
4,58532,0016C0F299261311C9AD0D08788FDC0B35AF1D34,2,22,2,1984,7,5,5,1,1,NINGUNO,58532,262,4,34,2017-12-11,2017-12-11,133,1,1,1,None,36,1,1,2,1,2,NINGUNO,1,1,3,3,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142894,51415,FFF4BFD72BB34166BB4792F33635E074D9208D91,2,22,2,1985,6,4,5,1,1,NINGUNO,51415,42,4,142872,2016-08-27,2016-12-09,112,1,1,1,None,35,1,1,2,1,2,NINGUNO,1,1,3,3,1,1
142895,71840,FFF5368EAC547E0EC82267C6B4A36E125BB23391,1,22,2,1958,1,4,5,1,1,NINGUNO,71840,137,6,142873,2019-07-05,2019-07-08,81,1,1,1,None,61,1,1,2,1,2,NINGUNO,1,1,1,1,1,1
142896,55314,FFF7ADD27607233E4A4D1BCF0689EEE1812E1727,2,22,2,1996,6,2,5,1,1,NINGUNO,55314,145,4,142876,2017-06-09,2017-07-12,13,1,2,2,None,23,1,2,1,1,2,NINGUNO,1,1,3,3,1,1
142897,50172,FFF9F0A1B5F522FF4B29102FEDAE087EC6380416,2,22,2,1962,3,9,5,1,1,NINGUNO,50172,232,5,142882,2011-03-31,2016-10-01,113,1,1,1,2018-01-24,58,1,2,2,1,2,NINGUNO,2,1,3,3,1,1


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142899 entries, 0 to 142898
Data columns (total 36 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   id_persona              142899 non-null  int64 
 1   internoen               142899 non-null  object
 2   genero                  142899 non-null  int64 
 3   nacionalidad            142899 non-null  int64 
 4   reincidente             142899 non-null  int64 
 5   anio_nacimiento         142899 non-null  int64 
 6   estado_civil            142899 non-null  int64 
 7   nivel_educativo         142899 non-null  int64 
 8   reconocimiento_etnico   142899 non-null  int64 
 9   diversidad_sexual       142899 non-null  int64 
 10  extranjero              142899 non-null  int64 
 11  condicion_exepcional    142899 non-null  object
 12  persona_id_persona      142899 non-null  int64 
 13  delito_id_delito        142899 non-null  int64 
 14  estado_ingreso          142899 non-n

### One-Hot Encoding

#### Evaluating Real number of intern with its unique booking date

In [24]:
df.groupby(['internoen','fecha_ingreso'])[['delito_id_delito']].count()

,,delito_id_delito
internoen,fecha_ingreso,
000334541B6B3305CE91761942D7B381248777EA,2018-08-06,1
000357E4F19E3722BA43488F69A7D7DECFC620C4,2019-09-25,2
000393AD7FE65FBBE53A61B0EAA1FB9A62F71450,2013-05-17,2
0003BE212427DF2E9CCB2449B6012EF915B3FAE3,2014-10-24,1
00043EB254F6E5EFCE77A8C20BC709B63A4BE443,2018-08-23,1
...,...,...
FFFC9F1A5227DDD6F78179576170997CAAB63AD7,2016-04-14,1
FFFD204514A5DF9EE2F1E36CDBF16F0B79C45590,2015-12-24,1
FFFE667F85CF452DB70B955609103CB72ECA3EEB,2020-01-14,1


#### Calculating again delito pareto

In [29]:
counts = df['delito_id_delito'].value_counts().to_frame()
counts['%'] = (counts['delito_id_delito'] / counts['delito_id_delito'].sum())*100
counts['cum_%'] = (counts['delito_id_delito'].cumsum() / counts['delito_id_delito'].sum())*100
counts

,delito_id_delito,%,cum_%
145,53145,37.190603,37.190603
232,23923,16.741195,53.931798
105,16816,11.767752,65.699550
42,9864,6.902777,72.602328
141,8897,6.226076,78.828403
102,2832,1.981819,80.810223
109,2229,1.559843,82.370066
262,2169,1.517855,83.887921
172,2105,1.473068,85.360989
103,1798,1.258231,86.619221


#### Converting the no pareto delitos, into an united delito

In [34]:
df.loc[df[~df['delito_id_delito'].isin(counts.index[:10].values)].index, 'delito_id_delito'] = 'other'

#### One-hot encoding for delito, tentatiev, agravado and calificado

In [70]:
one_hot = pd.concat([df[['internoen','fecha_ingreso']],
                     pd.get_dummies(df.delito_id_delito, prefix='delito_id_delito'),
                     pd.get_dummies(df.tentativa, prefix='tentativa'),
                     pd.get_dummies(df.agravado, prefix='agravado'),
                     pd.get_dummies(df.calificado, prefix='calificado')],
                     axis = 1)
one_hot = one_hot.groupby(['internoen','fecha_ingreso']).sum().reset_index()
one_hot

,internoen,fecha_ingreso,delito_id_delito_42,delito_id_delito_102,delito_id_delito_103,delito_id_delito_105,delito_id_delito_109,delito_id_delito_141,delito_id_delito_145,delito_id_delito_172,delito_id_delito_232,delito_id_delito_262,delito_id_delito_other,tentativa_1,tentativa_2,agravado_1,agravado_2,calificado_1,calificado_2
0,000334541B6B3305CE91761942D7B381248777EA,2018-08-06,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1,0
1,000357E4F19E3722BA43488F69A7D7DECFC620C4,2019-09-25,1,0,0,0,0,0,1,0,0,0,0,2,0,1,1,1,1
2,000393AD7FE65FBBE53A61B0EAA1FB9A62F71450,2013-05-17,0,0,0,1,0,0,1,0,0,0,0,2,0,1,1,1,1
3,0003BE212427DF2E9CCB2449B6012EF915B3FAE3,2014-10-24,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1,0
4,00043EB254F6E5EFCE77A8C20BC709B63A4BE443,2018-08-23,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103285,FFFC9F1A5227DDD6F78179576170997CAAB63AD7,2016-04-14,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1,0
103286,FFFD204514A5DF9EE2F1E36CDBF16F0B79C45590,2015-12-24,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,1,0
103287,FFFE667F85CF452DB70B955609103CB72ECA3EEB,2020-01-14,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,1,0
103288,FFFF73D0B7088A0EF0A4B7F3CC26F8CF672C6EC1,2016-04-05,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1


#### Removing non important id columns, and columns realated with the hot encoding

In [79]:
col = df.columns.tolist()
col.remove('delito_id_delito')
col.remove('id_registro')
col.remove('id_persona')
col.remove('persona_id_persona')
col.remove('condicion_exepcional')
col.remove('tentativa')
col.remove('agravado')
col.remove('calificado')
col.remove('fecha_captura')

#### Droping duplicates and merging with the one_hot df

In [82]:
df_model = df[col].drop_duplicates().merge(one_hot, on = ['internoen','fecha_ingreso'])

In [83]:
df_model

,internoen,genero,nacionalidad,reincidente,anio_nacimiento,estado_civil,nivel_educativo,reconocimiento_etnico,diversidad_sexual,extranjero,estado_ingreso,fecha_ingreso,establecimiento,fecha_salida,edad,municipio_id_municipio,actividades_estudio,actividades_trabajo,actividades_enseñanza,hijos_menores,condicion_excepcional,estado_id_estado,situacion_juridica,madre_gestante,madre_lactante,discapacidad,adulto_mayor,delito_id_delito_42,delito_id_delito_102,delito_id_delito_103,delito_id_delito_105,delito_id_delito_109,delito_id_delito_141,delito_id_delito_145,delito_id_delito_172,delito_id_delito_232,delito_id_delito_262,delito_id_delito_other,tentativa_1,tentativa_2,agravado_1,agravado_2,calificado_1,calificado_2
0,0008649DEF36F510807D0AB58A84CF74E6083FBF,2,22,2,1979,7,2,5,1,1,4,2019-09-02,14,None,40,1,1,2,1,2,NINGUNO,1,1,3,3,1,1,1,0,0,0,0,0,1,0,0,0,0,2,0,1,1,2,0
1,0008649DEF36F510807D0AB58A84CF74E6083FBF,2,22,2,1979,7,2,5,1,1,2,2016-01-29,16,2017-02-02,40,1,1,2,1,2,NINGUNO,2,1,3,3,1,1,1,0,0,0,0,0,1,0,0,0,0,2,0,2,0,2,0
2,0016C0F299261311C9AD0D08788FDC0B35AF1D34,2,22,2,1984,7,5,5,1,1,4,2017-12-11,133,None,36,1,1,2,1,2,NINGUNO,1,1,3,3,1,1,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0
3,001FEE5A33538FF7A1230A92CE34D5ADDEFCB31F,2,22,2,1983,3,2,5,1,1,4,2011-02-02,131,2012-05-23,36,1,2,2,1,2,NINGUNO,2,1,3,3,1,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1,0
4,00211F83BE5A06272B94EA85087548FA78D4142B,2,22,2,1963,3,2,5,1,1,4,2011-07-14,88,2013-08-20,56,1,2,1,1,2,NINGUNO,2,1,3,3,1,1,0,0,0,1,0,0,1,0,0,0,0,2,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103285,FFF37A09854265D8FDA4A47D933B1E980A310CD3,2,22,2,1990,7,3,5,1,1,4,2018-05-09,31,None,29,1,2,1,1,1,NINGUNO,1,1,3,3,1,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1,0
103286,FFF4BFD72BB34166BB4792F33635E074D9208D91,2,22,2,1985,6,4,5,1,1,4,2016-12-09,112,None,35,1,1,2,1,2,NINGUNO,1,1,3,3,1,1,1,0,0,0,0,0,0,0,1,0,1,3,0,3,0,3,0
103287,FFF5368EAC547E0EC82267C6B4A36E125BB23391,1,22,2,1958,1,4,5,1,1,6,2019-07-08,81,None,61,1,1,2,1,2,NINGUNO,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0
103288,FFF7ADD27607233E4A4D1BCF0689EEE1812E1727,2,22,2,1996,6,2,5,1,1,4,2017-07-12,13,None,23,1,2,1,1,2,NINGUNO,1,1,3,3,1,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1


#### Evaluating results

In [80]:
df_model.groupby(['internoen','fecha_ingreso'])[['anio_nacimiento']].count().sort_values(by = ['anio_nacimiento'], ascending = False)

,,anio_nacimiento
internoen,fecha_ingreso,
000334541B6B3305CE91761942D7B381248777EA,2018-08-06,1
AB3F8275473BBD8AF97B5139C9455738904B584D,2014-08-25,1
AB4486AC0515DDEE9F06D1BB1F59E1A6020D6B2E,2012-06-12,1
AB43D0F4513E1D28531433CD9D1EDFE08409C0B7,2020-05-13,1
AB4386380EC3F3385404C180E5121DD9D88E3D5D,2019-12-05,1
...,...,...
55C19F319053D12E41085E3FF4F28FB7D3005EC6,2011-10-24,1
55C1042A7931666EAC8AD3BC335A79F1F3D2B215,2012-07-23,1
55C038019C4B6BE5941F785C7ED7CC85BF165B29,2018-01-17,1


In [78]:
df[col][df[col]['internoen'] == '09E4B0900986C773B6820A305E5BB6438327E343']

,internoen,genero,nacionalidad,reincidente,anio_nacimiento,estado_civil,nivel_educativo,reconocimiento_etnico,diversidad_sexual,extranjero,estado_ingreso,fecha_captura,fecha_ingreso,establecimiento,fecha_salida,edad,municipio_id_municipio,actividades_estudio,actividades_trabajo,actividades_enseñanza,hijos_menores,condicion_excepcional,estado_id_estado,situacion_juridica,madre_gestante,madre_lactante,discapacidad,adulto_mayor
1368,09E4B0900986C773B6820A305E5BB6438327E343,2,22,2,1988,3,5,5,1,1,4,2020-02-24,2020-03-05,13,2020-03-16,31,1,2,2,1,1,NINGUNO,2,1,3,3,1,1
1369,09E4B0900986C773B6820A305E5BB6438327E343,2,22,2,1988,3,5,5,1,1,4,2017-05-17,2015-05-16,1,2017-09-22,31,1,2,2,1,1,NINGUNO,2,1,3,3,1,1
1370,09E4B0900986C773B6820A305E5BB6438327E343,2,22,2,1988,3,5,5,1,1,4,2017-05-26,2015-05-16,1,2017-09-22,31,1,2,2,1,1,NINGUNO,2,1,3,3,1,1
1371,09E4B0900986C773B6820A305E5BB6438327E343,2,22,2,1988,3,5,5,1,1,4,2017-03-03,2015-05-16,1,2017-09-22,31,1,2,2,1,1,NINGUNO,2,1,3,3,1,1
1372,09E4B0900986C773B6820A305E5BB6438327E343,2,22,2,1988,3,5,5,1,1,4,2014-01-28,2015-05-16,1,2017-09-22,31,1,2,2,1,1,NINGUNO,2,1,3,3,1,1
1373,09E4B0900986C773B6820A305E5BB6438327E343,2,22,2,1988,3,5,5,1,1,4,2016-05-16,2015-05-16,1,2017-09-22,31,1,2,2,1,1,NINGUNO,2,1,3,3,1,1
1374,09E4B0900986C773B6820A305E5BB6438327E343,2,22,2,1988,3,5,5,1,1,4,2017-01-17,2015-05-16,1,2017-09-22,31,1,2,2,1,1,NINGUNO,2,1,3,3,1,1
1375,09E4B0900986C773B6820A305E5BB6438327E343,2,22,2,1988,3,5,5,1,1,4,2015-12-03,2015-05-16,1,2017-09-22,31,1,2,2,1,1,NINGUNO,2,1,3,3,1,1
